# Equal Weight S&P 500 Index Fund

## Introduction
S&P 500 Project Overview 
The S&P 500 is the worlds most popular stock market index.

Many investment funds are benchmarked to the S&P 500. This means that they seek to replicate performance of this index by owning all the stocks that are held in the index. 

One of the most important characteristics of the S&P 500 is that it is market capitalization-weighted (its a big company).
This means that larger companies get a correspondingly larger weight in the index.

In the project, I will build an alternative version of the S&P 500 where each company has the same weight.

The python program will accept the value of a portfolio and tell me how many shares of each S&P 500 constituent i should purchase to get an equal-weight version of the index fund.

In [33]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

# Importing My List of Stocks
Here i will import the constituents of the S&P 500 into my notebook.



In [5]:
# i will start by saving the contents of the csv file into a dataframe.
# i do this by using the pandas method pd.read_csv()
stocks = pd.read_csv('sp_500_stocks.csv')

# Getting an API Token
I will import my IEX Cloud API token. This will be the data that i will use throughout this project.

API tokens should be stored in a secrets.py file that doesnt get pushed to my local git repo. I will be using a sandbox API token. This means that the data i use will be randomly generated and has no cost. 

In [6]:
from secrets import IEX_CLOUD_API_TOKEN

# Making My First API Call
Now i will structure my API calls to IEX cloud

I need the following info from the API:
    
    -Market cap for each stock
    -Price of each stock

In [7]:
# to start i will make the call for only one stock.
# then to generalize that for the rest of the stocks i will loop through every stock that is contained-
# in the pandas dataframe i created earlier. then run that same API call for each of those stocks 

symbol = 'AAPL'

# here i will use the base url. using the 'f' function to pass in the AAPL string.
# then, i need to add an appendish that passes in my api token to authenticate me.(?token={token from secrets.py}) 
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'

# now i need to execute an http request to store the results of that http request inside a variable.
# i will call this variable data and use the requests.get() function and pass in the api_url.
# i need to tranform the data that the http request returned into a json object.
data = requests.get(api_url).json()
print(data)

# now this returned a python dictionary object with all sorts of data inside.
# the two attributes that i want out of it are market cap and price.
# i will parse that in the next section.

{'avgTotalVolume': 87451355, 'calculationPrice': 'close', 'change': 2.2, 'changePercent': 0.01353, 'close': 0, 'closeSource': 'oaiflfic', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 170.25, 'iexCloseTime': 1689586102416, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 169.18, 'iexOpenTime': 1715988186607, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1724719387298, 'latestPrice': 168.54, 'latestSource': 'Close', 'latestTime': 'February 25, 2022', 'latestUpdate': 1702182104562, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2718739321825, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTim

# Parsing My API Call
The API call that i executed in the last code block contains all of the information required to build my equal-weight S&P 500 strategy.

But right now my data is not in the proper format. i need to parse it first. 

In [8]:
# parsing for the price
price = data['latestPrice']

# parsing for the market cap
market_cap = data['marketCap']
print(market_cap)

2718739321825


# Adding My Stocks Data to a Pandas DataFrame 
The next thing i need to do is add my stock's price and make a market cap to a pandas DataFrame. Think of a DataFrame like the python version of a spreadsheet. It stores tabular data.  

In [9]:
# the first thing i will do is specify the columns of the pandas dataframe.
# these columns will be the ticker for each stock, stock price for each stock, market cap for each stock, number-
# of shares to buy for each stock

# this variable will contain a python list 
my_columns = ['Ticker', 'Stock Price', 'Market Cap', 'Number of Shares to Buy']

# here i will make a blank pandas dataframe that will have those columns specified 
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy


In [10]:
# now i will append the data points into the dataframe 
# to do this i will use the .append() method and inside i need to make a pandas series

# inside the pandas series i will add a list with the symbol,price,market_cap,N/A
# the N/A is for the number of shares to buy. i cant calculate it until i pull all the metrics for each stock.
# for now it will be N/A and later  will update it.

# i need to finish the list off with the index = my_columns which tells the append method which columns to add-
# this data to.
# the ignore_index=True is put at the end of the series by convention. if not, python will complain.

final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2805532860.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,AAPL,168.54,2718739321825,N/A


# Looping Through The Ticker in My List of Stocks
Using the same logic i outlined above, i can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop.

In [11]:
# i need to loop through every ticker variable and make an API call for those stocks and then store those calls-
# in my pandas dataframe

# to start i will override my old dataframe variable with a new dataframe with the same columns as before
final_dataframe = pd.DataFrame(columns = my_columns)

# then i will create my for loop tha will make the API calls for the stocks.
# note that this process is very slow because making http requests in python is super slow. it makes the request-
# one at a time. later i wll try batch requests that allows for multiple requests at once.

for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index=True
    )

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

...

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/2451700220.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,136.31,41471556224,N/A
1,AAL,17.7,11654339717,N/A
2,AAP,208.01,13095430200,N/A
3,AAPL,165.1,2787023238900,N/A
4,ABBV,155.97,272577129789,N/A
...,...,...,...,...
462,URI,319.64,23393437508,N/A
463,USB,57.46,87252440876,N/A
464,V,221.09,499971736525,N/A
465,VAR,184.05,16763929880,N/A


# Using Batch API Calls to Improve Performance
Batch API calls are one of the easiest ways to improve the performance of your code.

IEX Cloud limits their batch API calls to 100 tickers per request. This reduces the number of API calls frome 500 to 5. 

In this section i will split my list of stocks into groups of 100 and then make a batch API call for each one.

In [17]:
# got this code from StackOverflow
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [18]:
# the list i want to split is stocks
# the reason i pass it in as Ticker is because that is a pandas dataframe and Ticker is the header of the only column
# by including both stocks and ticker i get back a series
# to get this into a list i pass in the list function
symbol_groups = list(chunks(stocks['Ticker'], 100))

# this will be a list of strings where each string is a comma separated string of all the stocks in the symbol_groups-
# variable
symbol_strings = []

# now i need a for loop that loops through every panda series within the list, executes a batch API call,-
# and for every stock in that list appends the info from the stock to my final dataframe
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i])) 
    
final_dataframe = pd.DataFrame(columns = my_columns)

# the next thing i need to do is loop through every string in my strings object and then use that string to execute-
# the batch API request
# once i get to the json portion of the code, the next thing is parsing the data.

# more specifically i need to parse the data for every specific stock thats in this symbol string.
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in data:
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index=my_columns
            ),
            ignore_index=True
        )


final_dataframe

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/11/4vb1s2tn0wdgxlw91mdp_j2w0000gn/T/ipykernel_1032/1698912614.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,135.52,41763000248,N/A
1,AAL,18.3,11469546902,N/A
2,AAP,214.44,12790376783,N/A
3,AAPL,171.08,2772864098772,N/A
4,ABBV,151.08,271328995845,N/A
...,...,...,...,...
499,YUM,126.27,36300642697,N/A
500,ZBH,129.9,27287652568,N/A
501,ZBRA,437.34,22834420539,N/A
502,ZION,74.31,11303662339,N/A


# Calculating the Number of Shares to Buy
As i can see in the DataFrame above, i still have not calculated the number of shares of each stock to buy.

I will do that here.

In [19]:
# i want this to work no matter how big a portfolio is. 
# the way i will handle that is by creating a python input that says how large is the portfolio? based on what i tell-
# python it will calculate the number of shares to buy.

# here i will use the input function and ask for the portfolio value.
portfolio_size = input('Enter the value of your portfolio:')

# i need to make a try/accept statement in case the input is not a number.
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:1000000


In [21]:
# now that i have accepted the value from the user, i need to use that number to base my calculations on.
# since this is equal weight, every stock in the portfolio will have the same position size. 
# position size is a fancy finance term that means how much money you invest in each stock.

# since there 500 stocks an easy example is that if your portfolio is 505 thousand dollars all you would do is invest-
# 1000 dollars in each stock. 

# this is the easiest way to calculate that 
position_size = val/len(final_dataframe.index)

# now that i have the position size i need to calculate how many shares of each stock need to be purchesed to-
# get to my desired position size in that stock.
# i will need to iterate through my dataframe for each of the stocks

# 
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,135.52,41763000248,14
1,AAL,18.3,11469546902,108
2,AAP,214.44,12790376783,9
3,AAPL,171.08,2772864098772,11
4,ABBV,151.08,271328995845,13
...,...,...,...,...
499,YUM,126.27,36300642697,15
500,ZBH,129.9,27287652568,15
501,ZBRA,437.34,22834420539,4
502,ZION,74.31,11303662339,26


# Formatting My Excel Output
I will be using the xlsxWriter library for Python to make a nicely formatted excel file.

This is useful because i can automate this task and have this program run and then send my trading team the output so that they can buy and sell the stocks.

# Initializing my XlsxWriter Object

In [40]:
# the class i initialize is from the pandas library and not from xlsx writer. the reason for that is since pandas-
# deals with tabular data it integrates well with excel with no issues.
# the engine part of the code is used to specify that i want to use the xlsx module of the library.
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')

# the next step is passing my pandas dataframe into this object and specify which tab of the excel file i want it-
# saved to. 
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

# Creating the Formats I will need For My .xlsx File
Formats include colors, fonts, and also symbols like % and $. I will need four main formats for my Excel document.

    String format for ticker
    $XX.XX format for stock price
    $XX,XXX format for market cap
    Integer format for the number of shares to purchase

In [41]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

# Applying the Formats to the Columns of My .xlsx File

In [42]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Cap', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

In [43]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

# Saving the Excel Output

In [44]:
writer.save()